In [27]:
%%capture
!pip install plotly
!pip install alpha_vantage
!pip install quandl
!pip install --upgrade plotly
!pip install --upgrade notebook
!pip install arch
!pip install cvxpy
!pip install fredapi
%run config.py

In [28]:
from alpha_vantage.timeseries import TimeSeries
import quandl
import matplotlib.pyplot as plt
import config 
import plotly.express as px
import pandas as pd
import os
import numpy as np
import datetime
import plotly.io as pio
pio.renderers.default = 'notebook'
from arch import arch_model
import cvxpy as cp
from fredapi import Fred
def write_date_to_file(file_path):
    today_date = datetime.datetime.today().strftime("%Y-%m-%d")
    with open(file_path, "w") as file:
        file.write(today_date)

# Function to read the date from the file
def read_date_from_file(file_path):
    with open(file_path, "r") as file:
        saved_date = file.read().strip()
    return saved_date


In [86]:
#### Get inflation data

def get_macro_data(series_id):
    fred = Fred(api_key= api_fred)
    
    macro_data = fred.get_series(series_id)
    macro_data = pd.DataFrame(macro_data).reset_index()
    macro_data.columns=['date','cpi']
    macro_data['date'] = pd.to_datetime(macro_data['date'],format = '%Y-%m-%d')
    macro_data.set_index('date',inplace=True)
    macro_data = macro_data.resample('D').interpolate().reset_index()
    macro_data['cpi_growth'] = macro_data['cpi'].pct_change()
    return macro_data

df_cpi = get_macro_data('CPIAUCSL')


In [87]:
### Fetch stock data

date_path = 'last_fetched.txt'
subfolder='data/'
list_symbols = ['AAPL','AMZN','TSLA','VEQT.TO']

    #### Check if the data was recently updated.
ts = TimeSeries(key=config.api_key, output_format='pandas')

if not os.path.exists(date_path):
    write_date_to_file(date_path)
saved_date = read_date_from_file(date_path)
today_date = datetime.datetime.today().strftime("%Y-%m-%d")
# Compare the saved date and today's date
if saved_date < today_date:
    rewrite_data = True
    write_date_to_file(date_path)
else:
    rewrite_data = False


common_dates = pd.date_range(start='2016-01-01', end='2023-10-01', freq='D')

df_returns = pd.DataFrame({'date': common_dates})


for symbol in list_symbols:    
    data_path = os.path.join(subfolder, 'portfolio_data_' + symbol.replace('.','_') + '.parquet')
    print("rewriting data? : ",((not os.path.exists(data_path)) | rewrite_data))
    global data
    if ((not os.path.exists(data_path)) | rewrite_data):
        data, _ = ts.get_weekly_adjusted(symbol=symbol)
        
    else:
        data = pd.read_parquet(data_path)


    data = data.sort_values(by='date')
    data = data.resample('D').interpolate().reset_index()
    data = data[['date','5. adjusted close']]
    # Rename the '5. adjusted close' column to be symbol-specific
    data.rename(columns={'5. adjusted close': 'adj_close_' + symbol}, inplace=True)

    # Merge the symbol-specific data with the common date index using the 'date' column
    df_returns = pd.merge(df_returns, data, on='date', how='left')


df_final = pd.merge(df_returns,df_cpi,on = 'date',how='left').dropna()

rewriting data? :  False
rewriting data? :  True
rewriting data? :  False
rewriting data? :  False


,date,adj_close_AAPL,adj_close_AMZN,adj_close_TSLA,adj_close_VEQT.TO,cpi,cpi_growth
1141,2019-02-15,41.1195,80.397500,20.525300,24.2170,253.762000,0.000124
1142,2019-02-16,41.2074,80.566143,20.399871,24.2331,253.793429,0.000124
1143,2019-02-17,41.2953,80.734786,20.274443,24.2492,253.824857,0.000124
1144,2019-02-18,41.3832,80.903429,20.149014,24.2653,253.856286,0.000124
1145,2019-02-19,41.4711,81.072071,20.023586,24.2814,253.887714,0.000124
...,...,...,...,...,...,...,...
2796,2023-08-28,183.2600,135.342857,241.341429,35.7200,307.324613,0.000127
2797,2023-08-29,184.8100,136.037143,242.258571,35.8600,307.363710,0.000127
2798,2023-08-30,186.3600,136.731429,243.175714,36.0000,307.402806,0.000127
2799,2023-08-31,187.9100,137.425714,244.092857,36.1400,307.441903,0.000127


In [79]:
macro_data.head()

,date,cpi,cpi_growth
0,1947-01-01,21.480000,NaN
1,1947-01-02,21.484516,0.00021
2,1947-01-03,21.489032,0.00021
3,1947-01-04,21.493548,0.00021
4,1947-01-05,21.498065,0.00021
